In [1]:
import json
import pandas as pd
import numpy as np
import torch.utils.data as data
import networkx as nx
import nltk
import spacy
import gensim
import en_core_web_sm
from nltk.data import find
from nltk.corpus import wordnet

/Users/ramraj/.local/share/virtualenvs/sg-generation-6Q740Mlp/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
import os
import h5py

In [3]:
SRC_SG_PATH = "../VG-data/scene_graphs.json"

SRC_ANSWER_VOCAB_FILE = "./intermediate_files/answer_vocab.txt"
SRC_SGQAS_OF_INTEREST_QA_DATA_FILE = "./intermediate_files/filtered_qa_data.json"

DST_SG_FEATURES_DATA_FOLDER = "./intermediate_files/sg_features/"

In [4]:
global_sg_data = json.load(open(SRC_SG_PATH, 'r'))
global_qa_data = json.load(open(SRC_SGQAS_OF_INTEREST_QA_DATA_FILE, 'r'))

# Simple Dataset for GCN

In [10]:
from collections import OrderedDict


NO_OF_NODES = 30


class Vertex:
    def __init__(self, node):
        self.id = node
        self.adjacent = OrderedDict()

    def __str__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    def get_connections(self):
        return self.adjacent

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent[neighbor]
    

def read_embedding_of_pred(image_id):
    feature_filename = os.path.join(DST_SG_FEATURES_DATA_FOLDER, "{}.h5".format(image_id))
    return h5py.File(feature_filename, 'r'):

In [17]:
sample_img = global_sg_data[0]['relationships']

In [18]:
sample_img[0]

{'synsets': ['along.r.01'],
 'predicate': 'ON',
 'relationship_id': 15927,
 'object_id': 5046,
 'subject_id': 5045}

In [9]:
class Graph:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, cost=0):
        # Ensure that we only the maximum number of nodes(graph vertices)
        if frm not in self.vert_dict and (len(self.get_vertices()) < NO_OF_NODES):
            self.add_vertex(frm)
        if to not in self.vert_dict and (len(self.get_vertices()) < NO_OF_NODES):
            self.add_vertex(to)

        # Add the edge to the graph only if both the vertices are added
        if frm in self.vert_dict and to in self.vert_dict:
            self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)

    def get_vertices(self):
        return self.vert_dict.keys()

    def convert_to_adj(self, image_id, is_inference=False):
        embeddings_hf = read_sg_features(image_id)
        
        nodes = self.get_vertices()
        nodes_indices = {k: v for v, k in enumerate(nodes)}
        
        adj_mat = [[0 for j in range(NO_OF_NODES)] for k in range(NO_OF_NODES)]     
        adj_mat_wo_wts = [[0 for j in range(NO_OF_NODES)] for k in range(NO_OF_NODES)]
        adj_mat_inf = [[-1 for j in range(NO_OF_NODES)] for k in range(NO_OF_NODES)]
        for node in nodes:
            adj_list = self.vert_dict[node].get_connections()

            for adj_node in adj_list:

                adj_mat[nodes_indices[self.vert_dict[node].id]][nodes_indices[adj_node.id]] = pred_embeddings[
                    adj_list[adj_node]]

                adj_mat_wo_wts[nodes_indices[self.vert_dict[node].id]][nodes_indices[adj_node.id]] = 1
                if is_inference:
                    adj_mat_inf[nodes_indices[self.vert_dict[node].id]][nodes_indices[adj_node.id]] = adj_list[adj_node]

                # adj_mat[nodes_indices[self.vert_dict[node].id]][nodes_indices[adj_node.id]] = adj_list[adj_node]

        # print(np.asarray(adj_mat).shape, "\t\t : \t\t", np.asarray(vis_adj_mat).shape)
        if is_inference:
            return adj_mat, vis_adj_mat, adj_mat_wo_wts, nodes_indices, adj_mat_inf
        return adj_mat, vis_adj_mat, adj_mat_wo_wts, nodes_indices
    

def parse_rel_annotation(scene_graph_data):
    
    img_to_graph = {}
    for img in img_ids:
        g = Graph()
        num = 0
        
        for rel in scene_graph_data['relationships']:
            g.add_edge(rel['subject_id'], rel['object_id'], cost=1)
            
        img_to_graph[img] = g

    return img_to_graph


def generate_graph(scene_graph_data, is_inference=False):

    adj_mats_dict = {}
    adj_mats_wo_wts_dict = {}
    nodes_indices_dict = {}
    adj_mat_inf_dict = {}

    img_to_SG_graph = parse_rel_annotation(scene_graph_data)
    nodes_indices = {}
    bias_mat_dict = {}
    for SG_graph in img_to_SG_graph:
        
        adj_mat, adj_mat_wo_wts, nodes_indices = SG_graph[img].convert_to_adj()

        adj_mats_dict[img] = adj_mat
        adj_mats_wo_wts_dict[img] = adj_mat_wo_wts
        nodes_indices_dict[img] = nodes_indices

    return adj_mats_dict, adj_mats_wo_wts_dict, nodes_indices_dict


class VQAVGSimpleDataset(data.Dataset):

    def __init__(self, sg_data_path, qa_data_path, ans_vocab_data_path, feature_path):
        self.sg_data_path = sg_data_path
        self.qa_data_path = qa_data_path
        self.ans_vocab_data_path = ans_vocab_data_path  
        self.feature_path = feature_path
        
        self.sample_cnt = 0
        self.data_sgvqa = []
        self._load_dataset()
        
    def _load_dataset(self):
        print('-> Loading filtered dataset ...')
        self.ans_vocab_data = sorted(open(self.ans_vocab_data_path, 'r').read().strip().split("\n"))
        sg_data = global_sg_data[:10]
        qa_data = global_qa_data[:10]
        
        generate_graph(sg_data)
        
final_dataset = VQAVGSimpleDataset()

In [5]:
from scipy.sparse import csgraph
import scipy.sparse as sp
import torch


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    
def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)

    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt).tocoo()

In [93]:
class VQAVGSimpleDataset(data.Dataset):

    def __init__(self, sg_data_path, qa_data_path, ans_vocab_data_path, feature_path):
        self.sg_data_path = sg_data_path
        self.qa_data_path = qa_data_path
        self.ans_vocab_data_path = ans_vocab_data_path  
        self.feature_path = feature_path
        
        self.sample_cnt = 0
        self.data_sgvqa = []
        self._load_dataset()
        
    def _load_dataset(self):
        print('-> Loading filtered dataset ...')
        self.ans_vocab_data = sorted(open(self.ans_vocab_data_path, 'r').read().strip().split("\n"))
        sg_data = global_sg_data[:10]
        qa_data = global_qa_data[:10]
#         sg_data = json.load(open(self.sg_data_path, 'r'))
#         qa_data = json.load(open(self.qa_data_path, 'r'))            
        
        for sample_img, sample_ans in zip(sg_data, qa_data):
            if sample_img['image_id'] != sample_ans['id']:
                print("IDs did not match !")
                continue                
                
            feature_filename = os.path.join(DST_SG_FEATURES_DATA_FOLDER, "{}.h5".format(sample_img['image_id']))
            with h5py.File(feature_filename, 'r') as hf:
                
                g = nx.Graph()
                feature_matrix = []
                for obj in sample_img['objects']:
                    obj_name = obj['names'][0]
                    obj_id = obj['object_id']                    
                    emb_vec = np.array(hf.get(str(obj_id)))

                    g.add_node(obj_id, feature=emb_vec)
                    feature_matrix.append(emb_vec)
                        
                for rel in sample_img['relationships']:
                    g.add_edge(rel['subject_id'], rel['object_id'], id=rel['relationship_id'])

                adj = nx.adjacency_matrix(g)
                # print(adj.todense())
                adj = normalize_adj(adj + sp.eye(adj.shape[0]))
                sparse_mx = adj.tocoo().astype(np.float32)
                # adj = torch.FloatTensor(np.array(adj.todense()))
                adj = np.array(adj.todense())        
                
                # feature_matrix = normalize(np.asarray(feature_matrix))
                # feature_matrix = torch.FloatTensor(np.array(feature_matrix.todense()))
#                 print(np.array(feature_matrix, np.float32).shape)
#                 feature_matrix = torch.FloatTensor(np.array(feature_matrix, np.float32))
                
                
                adj_matrix_per_sample = []    
                feature_matrix_per_sample = []
                for qa_index, qa in enumerate(sample_ans['qas']):
                    if (qa['qas_skip']): continue
                    question = qa['question']
                    answer = qa['answer'].replace(".", "").lower()
                    # sg = sample_img  
                    
                    adj_matrix_per_sample.append(adj)
                    feature_matrix_per_sample.append(feature_matrix)
                
                print(len(feature_matrix_per_sample))
                print(len(adj_matrix_per_sample))
                print(len(feature_matrix_per_sample[0]))
                print(len(adj_matrix_per_sample[0]))
                print(adj_matrix_per_sample[0].shape)
                f = np.array(feature_matrix_per_sample, np.float32)
                a = np.array(adj_matrix_per_sample)
                print(f.shape)
                print(a.shape)
                feature_matrix_per_sample = torch.FloatTensor(np.array(feature_matrix_per_sample, np.float32))
                adj_matrix_per_sample = torch.FloatTensor(np.array(adj_matrix_per_sample))
                
                self.data_sgvqa.append({"question": question, "answer": answer,
                                        "sg_adj": adj_matrix_per_sample, "sg_feat": feature_matrix})
                
                self.sample_cnt += 1
                if (self.sample_cnt > 10): break # todo: remove                       
        
        print('-> Finished loading data : num. samples -> {}'.format(self.sample_cnt))
        
    def __len__(self):
        return self.sample_cnt
                
    def __getitem__(self, index):
        if index < self.sample_cnt:
            item = self.data_sgvqa[index]
        else:
            item = self.data_sgvqa[index - self.sample_cnt]
        return item

    def num_classes(self):
        return len(self.ans_vocab_data)

#     def vocab_words(self):
#         return self.dataset_vqa.vocab_words()

#     def vocab_answers(self):
#         return self.dataset_vqa.vocab_answers()

    def data_loader(self, batch_size=10, num_workers=4, shuffle=False):
        return DataLoader(self, 
                          batch_size=batch_size, 
                          shuffle=shuffle,
                          num_workers=num_workers, 
                          pin_memory=True)

    def split_name(self, testdev=False):
        return self.data_sgvqa.split_name(testdev=testdev)

In [94]:
vqa_dataloader = VQAVGSimpleDataset(SRC_SG_PATH, SRC_SGQAS_OF_INTEREST_QA_DATA_FILE, SRC_ANSWER_VOCAB_FILE,
                                    DST_SG_FEATURES_DATA_FOLDER)

-> Loading filtered dataset ...
61
61
40
40
(40, 40)


ValueError: setting an array element with a sequence.